In [2]:
#Sentiment analysis dataset loader
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('sentiment_analysis.csv', encoding='latin-1')

In [6]:
df = df[['text', 'sentiment']]

In [9]:
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.replace('[^\w\s]', '', regex=True)
df['text'] = df['text'].str.replace('\d+', '', regex=True)
df['text'] = df['text'].str.strip()
df = df.dropna()

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\d'
/var/folders/4j/n0rl6b3s0657t_7z21qxxz6w0000gn/T/ipykernel_31351/2139294192.py:2: SyntaxWarning: invalid escape sequence '\w'
  df['text'] = df['text'].str.replace('[^\w\s]', '', regex=True)
/var/folders/4j/n0rl6b3s0657t_7z21qxxz6w0000gn/T/ipykernel_31351/2139294192.py:3: SyntaxWarning: invalid escape sequence '\d'
  df['text'] = df['text'].str.replace('\d+', '', regex=True)
/var/folders/4j/n0rl6b3s0657t_7z21qxxz6w0000gn/T/ipykernel_31351/2139294192.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()
/var/folders/4

In [10]:
df

,text,sentiment
0,id have responded if i were going,neutral
1,sooo sad i will miss you here in san diego,negative
2,my boss is bullying me,negative
3,what interview leave me alone,negative
4,sons of why couldnt they put them on the rele...,negative
...,...,...
27476,wish we could come see u on denver husband lo...,negative
27477,ive wondered about rake to the client has mad...,negative
27478,yay good for both of you enjoy the break you ...,positive
27479,but it was worth it,positive


In [12]:
#removing links 
import re
def remove_links(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
df.loc[:, 'text'] = df['text'].apply(remove_links)

In [18]:
df.iloc[428]

text         my name is tony not hey   poor tony
sentiment                               negative
Name: 429, dtype: object

In [19]:
#removing stops words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ankitpokhrel/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [20]:
#removing stopwords form text column
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

/var/folders/4j/n0rl6b3s0657t_7z21qxxz6w0000gn/T/ipykernel_31351/4127354768.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


In [22]:
df.iloc[428]

text         name tony hey poor tony
sentiment                   negative
Name: 429, dtype: object

In [23]:
#Stemming the words
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [24]:
#using stemming on text column
df['text'] = df['text'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))

/var/folders/4j/n0rl6b3s0657t_7z21qxxz6w0000gn/T/ipykernel_31351/4194091889.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))


In [25]:
df.iloc[428]

text         name toni hey poor toni
sentiment                   negative
Name: 429, dtype: object

In [26]:
#Label encoding the sentiment column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['sentiment'] = le.fit_transform(df['sentiment'])
df

/var/folders/4j/n0rl6b3s0657t_7z21qxxz6w0000gn/T/ipykernel_31351/683182114.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = le.fit_transform(df['sentiment'])


,text,sentiment
0,id respond go,1
1,sooo sad miss san diego,0
2,boss bulli,0
3,interview leav alon,0
4,son couldnt put releas alreadi bought,0
...,...,...
27476,wish could come see u denver husband lost job ...,0
27477,ive wonder rake client made clear net dont for...,0
27478,yay good enjoy break probabl need hectic weeke...,2
27479,worth,2


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
Cv=CountVectorizer(max_features=2500)

In [28]:
X = Cv.fit_transform(df['text']).toarray()
y = df['sentiment'].values

In [29]:
X.shape

(27480, 2500)

In [30]:
#Using train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
#Applying Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB,GaussianNB, BernoulliNB

In [32]:
#Fitting the all three models
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB()

In [33]:
#GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [34]:
#BernoulliNB
bnb = BernoulliNB()
bnb.fit(X_train, y_train)

BernoulliNB()

In [35]:
y_pred_mnb = mnb.predict(X_test)
y_pred_gnb = gnb.predict(X_test)
y_pred_bnb = bnb.predict(X_test)

In [36]:
#Calculating accuracy of all three models
from sklearn.metrics import accuracy_score
accuracy_mnb = accuracy_score(y_test, y_pred_mnb)
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
accuracy_bnb = accuracy_score(y_test, y_pred_bnb)
print("MultinomialNB Accuracy:", accuracy_mnb)
print("GaussianNB Accuracy:", accuracy_gnb)
print("BernoulliNB Accuracy:", accuracy_bnb)

MultinomialNB Accuracy: 0.644650655021834
GaussianNB Accuracy: 0.4306768558951965
BernoulliNB Accuracy: 0.654839883551674


In [37]:
#Testing with custom input
def predict_sentiment(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub('[^\w\s]', '', text)
    text = re.sub('\d+', '', text)
    text = text.strip()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = ' '.join([ps.stem(word) for word in text.split()])
    vectorized_text = Cv.transform([text]).toarray()
    prediction = mnb.predict(vectorized_text)
    sentiment = le.inverse_transform(prediction)
    return sentiment[0]

<>:5: SyntaxWarning: invalid escape sequence '\w'
<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\w'
<>:6: SyntaxWarning: invalid escape sequence '\d'
/var/folders/4j/n0rl6b3s0657t_7z21qxxz6w0000gn/T/ipykernel_31351/514224748.py:5: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('[^\w\s]', '', text)
/var/folders/4j/n0rl6b3s0657t_7z21qxxz6w0000gn/T/ipykernel_31351/514224748.py:6: SyntaxWarning: invalid escape sequence '\d'
  text = re.sub('\d+', '', text)


In [45]:
user_input = input("Enter a text to analyze sentiment: ")
print("You entered:", user_input)
print("Predicted Sentiment:", predict_sentiment(user_input))

You entered: I don't think that would be correct approach
Predicted Sentiment: neutral


In [47]:
user_input = input("Enter a text to analyze sentiment: ")
print("You entered:", user_input)
print("Predicted Sentiment:", predict_sentiment(user_input))

You entered: you are very bad at this
Predicted Sentiment: negative


In [48]:
user_input = input("Enter a text to analyze sentiment: ")
print("You entered:", user_input)
print("Predicted Sentiment:", predict_sentiment(user_input))

You entered: he is a good guy, you can trust him
Predicted Sentiment: positive
